# Based on [Optimization with CVXPY](https://towardsdatascience.com/optimization-with-python-how-to-make-the-most-amount-of-money-with-the-least-amount-of-risk-1ebebf5b2f29)

## Here, we will apply a basic optimization problem which could be close to the Skoda one, in a much simplified way though

### Note for Sarang
I guess it's not very difficult to go through this small notebook <br/>
I just want to let you know, that I had some problem with specifying the main constraint (capacity production)<br/>
I guess you can find the answer in the package documentation I shared on Teams. Anyway, we can discuss about this small example (which can be very close to Skoda one) anytime ofc

### Librairies and data creation

In [63]:
from teradataml import *
import pandas as pd
import cvxpy as cvx# TO DO: Look for a docker image, cause the installation could be very annoying
import random
import numpy as np

%matplotlib inline

In [32]:
random.seed(13)
df=pd.DataFrame({
    'Month':['201901','201901','201901','201901','201902','201902','201902','201902'],
     'Model (i)':['M1','M1','M2','M2','M1','M1','M2','M2'],
     'Country':['France','Germany','France','Germany','France','Germany','France','Germany'],
     'Quantity (j)': random.sample(range(1500,7400), 8),
     'Price':[12000,11000,18000,17000,12000,11000,18000,17000]
      })
df

,Month,Model (i),Country,Quantity (j),Price
0,201901,M1,France,3621,12000
1,201901,M1,Germany,3881,11000
2,201901,M2,France,7113,18000
3,201901,M2,Germany,7103,17000
4,201902,M1,France,3021,12000
5,201902,M1,Germany,6841,11000
6,201902,M2,France,3388,18000
7,201902,M2,Germany,6957,17000


### Optimization problem
We want to find the optimal sales (_Quantity_) according to some constraints: <br/>
>  Fixed prices<br/>
   Capacity production (Sum of quantities per month < Q)<br/>
   Minimun sales (Quantities per month > q)<br/>
   Market criteria : minimun sales per month X model X country <br/>
   
In the mathematical way, <br/>

> \begin{align*}
Max_{x_{i,j}} \sum_ {i,j}p_{i,j}x_{i,j}
\end{align*}
subject to:
\begin{align*}
\sum_ {i,j}x_{i,j} < K\\
\forall_{i,j}\enspace  q_{i,j}<x_{i,j}<Q_{i,j}
\end{align*}

#### After data we can review our dataset - Indeed, we are interest in 4 variables (2 Models X 2 countries)


In [76]:
new_df=df.drop(columns=['Month', 'Quantity (j)'])
new_df=new_df.drop_duplicates(['Model (i)','Country'],keep= 'last')
new_df

,Model (i),Country,Price
4,M1,France,12000
5,M1,Germany,11000
6,M2,France,18000
7,M2,Germany,17000


In [90]:
# Adding the variables to optimize to this new DF
inc=[]
for w in range(0,new_df.shape[0]):
    inc.append(cvx.Variable())
new_df['var'] = inc
new_df.reset_index(inplace=True)
new_df

,Model (i),Country,Price,var
4,M1,France,12000,var22
5,M1,Germany,11000,var23
6,M2,France,18000,var24
7,M2,Germany,17000,var25


#### Constraints

In [205]:
K=8000
Q=[3000,2500,2000,1500]
q=list(2*np.divide(Q,10))

new_df['min']=q
new_df['max']=Q


constraints=[cvx.sum([new_df['var']])<=K]
for w in range(0,new_df.shape[0]) :
    constraints.append([new_df['var'][w]<=new_df['max'][w]])
    constraints.append([new_df['var'][w]>=new_df['min'][w]])

#### Problem instance

In [207]:
obj = cvx.Maximize(cvx.sum([new_df['Price'][w]*new_df['var'][w] for w in range(0,new_df.shape[0]-1)]))

prob = cvx.Problem(obj)

In [2]:
val=prob.solve()